In [ ]:
!pip install langchain
!pip install chromadb

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from config import openaiapikey as key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = key

In [ ]:
template = """ You are going to be my assistant.
Please try to give me the most beneficial answers to my
question with reasoning for why they are correct.

 Question: {input} Answer: """
prompt = PromptTemplate(template=template, input_variables=["input"])

In [ ]:
chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0.8))

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
file = open("./sample_blog.txt", "r")
text = file.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)


In [ ]:
# pass the text and embeddings to ChromaDB
db = Chroma.from_texts(texts, embeddings,metadatas=[{"source": str(i)} for i in range(len(texts))])

In [ ]:
my_chain = load_qa_with_sources_chain(OpenAI(temperature=0.8), chain_type="refine", verbose=True)
query = "All places in Kochi"
documents = db.similarity_search(query)
que = "Make an itinerary with all the places mentioned along with their descriptions in one or two sentences."
result = my_chain({"input_documents": documents, "question": que})
result

In [ ]:
print(result['output_text'])